In [ ]:
import pandas as pd
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate, Bidirectional, GRU 
from keras import Model
from keras import optimizers
from tensorflow import keras
import time

In [ ]:
data_full = pd.read_csv('Intrahour Volatility Dataset.csv')

In [ ]:
X = data_full[["Return_Squared", "Hourly Volatility"]]
Y = data_full["target"]
data_set = data_full[["Date","Return_Squared", "Hourly Volatility", "target"]]

In [ ]:
splitlimit = int(len(data_full)*0.8)
training_features, test_data = data_set[:splitlimit], data_set[splitlimit:]

In [ ]:
#Outlier Detection in training_data_features

training_features["hourly_volatility_rolling_median"] = training_features["Hourly Volatility"].rolling(window=41, center=True, min_periods=1).median()
training_features["return_squared_rolling_median"] = training_features["Return_Squared"].rolling(window=41, center=True, min_periods=1).median()
training_features["volatility minus median"] = (training_features["Hourly Volatility"] - training_features["hourly_volatility_rolling_median"]).abs()
training_features["return minus median"] = (training_features["Return_Squared"] - training_features["return_squared_rolling_median"]).abs()
volatility_outliers_removed = training_features[~(training_features['volatility minus median'] > 5 * training_features['volatility minus median'].median())]
both_outliers_removed = volatility_outliers_removed[~(volatility_outliers_removed['return minus median'] > 5 * volatility_outliers_removed['return minus median'].median())]

In [ ]:
X_cleaned = both_outliers_removed[["Return_Squared", "Hourly Volatility"]]
Y_cleaned = both_outliers_removed["target"]
data_set_cleaned = both_outliers_removed[["Return_Squared", "Hourly Volatility", "target"]]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
training_data_features_scaled = scaler.fit_transform(X_cleaned)
data_set_scaled = scaler.fit_transform(data_set_cleaned)

In [ ]:
#Reconstructing training data
Z = []

backcandles = 15

for j in range(2):
    Z.append([])
    for i in range(backcandles, training_data_features_scaled.shape[0]):
        Z[j].append(training_data_features_scaled[i-backcandles:i, j])
        
Z = np.moveaxis(Z, [0], [2])    
Z, yi = np.array(Z), np.array(data_set_scaled[backcandles-1:, -1])
y_final = np.reshape(yi,(len(yi),1))
y_final = y_final[1:]

In [ ]:
lstm_input = Input(shape = (backcandles, 2), name = 'lstm_input')

inputs = LSTM(80, name='first_layer')(lstm_input)

inputs = Dense(1, name='dense_layer')(inputs)

output = Activation('sigmoid', name = 'output')(inputs)

model = Model(inputs = lstm_input, outputs = output)
model.compile(optimizer="adam", loss = "binary_crossentropy", metrics = ["accuracy"])
start_time = time.time()
history = model.fit(x=Z, y=y_final, epochs = 20, validation_data = (Z, y_final))
end_time = time.time()
training_time = end_time - start_time

In [ ]:

# Evaluate the model on the test data
training_accuracy = model.evaluate(Z)

print(training_time)
print(training_accuracy)

In [ ]:
gru_input = Input(shape = (backcandles, 2), name = 'gru_input')

inputs = GRU(80, name='first_layer')(gru_input)

inputs = Dense(1, name='dense_layer')(inputs)

output = Activation('sigmoid', name = 'output')(inputs)

model2 = Model(inputs = gru_input, outputs = output)
model2.compile(optimizer="adam", loss = "binary_crossentropy", metrics = ["accuracy"])
start_time2 = time.time()
history2 = model2.fit(x=Z, y=y_final, epochs = 20, validation_data = (Z, y_final))
end_time2 = time.time()
training_time2 = end_time2 - start_time2

In [ ]:
model2.summary()
print(training_time2)

In [ ]:
X_test = test_data[["Return_Squared", "Hourly Volatility"]]
Y_test = test_data["target"]
test_dataset = test_data[["Return_Squared", "Hourly Volatility", "target"]]

In [ ]:
#Scaling test data
test_scaled = scaler.fit_transform(test_dataset)
X_test_scaled = scaler.fit_transform(X_test)

In [ ]:
#Reconstructing test data
T = []

backcandles = 15

for j in range(2):
    T.append([])
    for i in range(backcandles, X_test_scaled.shape[0]):
        T[j].append(X_test_scaled[i-backcandles:i, j])
        
T = np.moveaxis(T, [0], [2])
T, yi_test = np.array(T), np.array(test_scaled[backcandles-1:, -1])
y_final_test = np.reshape(yi_test,(len(yi_test),1))
y_final_test = y_final_test[1:]        

In [ ]:
test_predictions1 = model.predict(T)
test_predicted_classes1 = (test_predictions1 > 0.5).astype(int)

In [ ]:
#lstm
test_predictions1 = model.predict(T)
test_predicted_classes1 = (test_predictions1 > 0.5).astype(int)
dataframe = pd.DataFrame(y_final_test, columns = ["target"])
dataframe["predicted"] = test_predicted_classes1
cm_lstm = confusion_matrix(dataframe['predicted'], dataframe['target'])
print(cm_lstm)

In [ ]:
#gru
test_predictions2 = model2.predict(T)
test_predicted_classes2 = (test_predictions2 > 0.5).astype(int)
dataframe2 = pd.DataFrame(y_final_test, columns = ["target"])
dataframe2["predicted"] = test_predicted_classes2
cm_gru = confusion_matrix(dataframe2['predicted'], dataframe2['target'])
print(cm_gru)